In [11]:
import pandas as pd

##Load one example: 
sample_path = '/Users/liuzehan/Desktop/STATS220 ML in Neuro/Project/main/data2/fullDLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070_filtered.csv'

df = pd.read_csv(sample_path)

,scorer,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.1,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.2,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.3,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.4,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.5,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.6,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.7,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.8,...,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.20,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.21,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.22,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.23,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.24,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.25,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.26,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.27,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.28,DLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.29
0,bodyparts,nose,nose,nose,left_ear,left_ear,left_ear,right_ear,right_ear,right_ear,...,right_leg,tail_base,tail_base,tail_base,tail_mid,tail_mid,tail_mid,tail_tip,tail_tip,tail_tip
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,...,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,77.37103,84.16902,0.4345222,67.41314,102.81786,0.57197803,88.58426,89.80255,0.4298043,...,0.3277821,141.1829,180.43906,0.46197087,181.29254,212.58356,0.7727549,229.55302,256.72232,1.0
3,1,78.35937,84.16902,0.41802916,67.41314,102.81786,0.60118705,88.677185,89.80255,0.45551544,...,0.50861984,143.23474,180.43906,0.63736176,182.39333,212.58356,0.8054246,232.1428,256.72232,0.9677174
4,2,78.35937,84.16902,0.38181394,69.00627,102.81786,0.58181673,88.677185,89.80255,0.42170873,...,0.43355042,144.97609,180.43906,0.6933703,185.169,212.58356,0.7662135,238.25171,256.72232,0.94790804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,2325,430.805,445.41562,0.5658416,425.9361,431.11993,0.6374442,413.9786,454.96466,0.682118,...,0.36578646,425.55197,440.47397,0.28037843,314.52646,340.0778,0.25327307,295.5463,275.44,0.71679735
2328,2326,433.62866,445.5089,0.57089907,427.64072,430.7673,0.71749574,416.36514,454.96466,0.6399309,...,0.37294033,428.27408,440.557,0.30793467,319.5943,339.45993,0.24650933,299.05774,276.00867,0.6596501
2329,2327,435.77542,445.5089,0.61019176,429.17535,429.58286,0.7495789,418.71942,454.96466,0.6693428,...,0.34627962,431.8459,438.7668,0.33351347,321.46198,339.45993,0.18342268,301.60608,276.00867,0.57915807
2330,2328,435.77542,445.5089,0.58698255,429.17535,428.7282,0.76472646,418.71942,453.6013,0.6779553,...,0.33703366,431.8459,438.7668,0.3442618,321.46198,339.45993,0.24333781,301.60608,276.00867,0.54330677


In [12]:
def preprocess_dlc_csv(df):
    # Select relevant data
    df_new = df.iloc[2:, 1:].reset_index(drop=True)
    
    # Create column names from row 0 and 1
    col_names = list(df.iloc[0, :].astype(str) + '_' + df.iloc[1, :].astype(str))[1:]
    df_new.columns = col_names

    # Drop likelihood columns
    df_new.drop(columns=[
        'nose_likelihood', 'left_ear_likelihood', 'right_ear_likelihood','head_likelihood',
        'mid_body_likelihood', 'left_leg_likelihood', 'right_leg_likelihood', 'tail_base_likelihood', 'tail_mid_likelihood', 'tail_tip_likelihood'
    ], inplace=True)

    # Convert all remaining values to numeric
    df_new = df_new.apply(pd.to_numeric, errors='coerce')

    return df_new

preprocessed_df = preprocess_dlc_csv(df)

In [13]:
def egocentric_center(preprocessed_df):
    # Define center coordinates
    head_x = preprocessed_df['head_x']
    head_y = preprocessed_df['head_y']
    
    # Create a copy to avoid modifying the original DataFrame
    df_centered = preprocessed_df.copy()
    
    # Iterate over all columns
    for col in df_centered.columns:
        if col.endswith('_x'):
            df_centered[col] = df_centered[col] - head_x
        elif col.endswith('_y'):
            df_centered[col] = df_centered[col] - head_y
    
    df_centered.drop(columns=['head_x', 'head_y'], inplace=True)
    df_centered['head_x_original'] = head_x
    df_centered['head_y_original'] = head_y
    
    return df_centered

centered_df = egocentric_center(preprocessed_df)
centered_df

,nose_x,nose_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,mid_body_x,mid_body_y,left_leg_x,left_leg_y,right_leg_x,right_leg_y,tail_base_x,tail_base_y,tail_mid_x,tail_mid_y,tail_tip_x,tail_tip_y,head_x_original,head_y_original
0,-8.54187,-27.516500,-18.49976,-8.86766,2.671360,-21.88297,20.544116,21.469830,12.599760,55.53369,58.96820,31.06891,55.27000,68.75354,95.37964,100.89804,143.64012,145.03680,85.91290,111.68552
1,-7.55353,-27.516500,-18.49976,-8.86766,2.764285,-21.88297,21.082484,21.469830,12.845250,55.53369,60.11626,31.06891,57.32184,68.75354,96.48043,100.89804,146.22990,145.03680,85.91290,111.68552
2,-7.55353,-27.516500,-16.90663,-8.86766,2.764285,-21.88297,21.623940,21.469830,16.287555,55.53369,60.11820,31.06891,59.06319,68.75354,99.25610,100.89804,152.33881,145.03680,85.91290,111.68552
3,-8.18274,-26.138220,-18.03461,-6.98981,3.030490,-21.60358,21.983070,24.155110,19.276680,58.41607,60.47733,32.55996,60.20435,70.61950,102.02640,100.81741,159.05835,139.15261,85.55377,106.76905
4,-8.93563,-24.596376,-18.14063,-7.81279,2.397490,-20.29588,23.703730,23.667105,19.336435,58.47608,62.71239,33.66241,63.55310,68.90349,103.28015,97.54452,160.44858,137.44137,85.55377,102.76333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,22.47895,9.118040,17.61005,-5.17765,5.652550,18.66708,-23.611180,-23.617180,-47.496190,-3.04016,-9.85330,-42.55121,17.22592,4.17639,-93.79959,-96.21978,-112.77975,-160.85758,408.32605,436.29758
2326,24.44623,9.249840,18.45829,-5.49176,7.182710,18.70560,-22.114590,-23.266780,-44.278630,-3.00164,-9.47903,-43.53846,19.09165,4.29794,-89.58813,-96.79913,-110.12469,-160.25039,409.18243,436.25906
2327,24.80170,9.828330,18.20163,-6.09771,7.745700,19.28409,-21.032500,-21.964750,-43.847520,-1.71231,-9.81909,-42.95997,20.87218,3.08623,-89.51174,-96.22064,-109.36764,-159.67190,410.97372,435.68057
2328,24.80170,10.319940,18.20163,-6.46076,7.745700,18.41234,-21.032500,-21.473140,-43.847520,-1.22070,-9.81909,-42.87158,20.87218,3.57784,-89.51174,-95.72903,-109.36764,-159.18029,410.97372,435.18896


In [14]:
centered_df.to_csv('data2/centered_df.csv', index=False)

In [15]:
csv_path = f'/Users/liuzehan/Desktop/STATS220 ML in Neuro/Project/main/data2/train_video/full_trainDLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.csv'
store_path = f'data2/train_dataset/full_train_centered.csv'

df = pd.read_csv(csv_path)
preprocessed_df = preprocess_dlc_csv(df)
centered_df = egocentric_center(preprocessed_df)
centered_df.to_csv(store_path, index=False)


In [16]:
csv_path = f'/Users/liuzehan/Desktop/STATS220 ML in Neuro/Project/main/data2/test_video/full_testDLC_Resnet50_2-dlc-pose-trackingJun5shuffle2_snapshot_070.csv'
store_path = f'data2/test_dataset/full_test_centered.csv'

df = pd.read_csv(csv_path)
preprocessed_df = preprocess_dlc_csv(df)
centered_df = egocentric_center(preprocessed_df)
centered_df.to_csv(store_path, index=False)
